In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import glob

from sklearn.metrics import confusion_matrix
import itertools

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPool1D
from keras.initializers import random_uniform
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as SS

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D
from keras.optimizers import  SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
from keras.layers import SimpleRNN
#from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt

# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

# Importing the dataset

In [ ]:
dataset = pd.read_csv(r'...') # Use your path

In [ ]:
dataset.shape

In [ ]:
# Splitting dataset into features and labels.

labels = dataset['Label']
features = dataset.loc[:, dataset.columns != 'Label'].astype('float64')

In [ ]:
features.shape , labels.shape

In [ ]:
# using Robust Scalar to scale data 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [ ]:
scaler = RobustScaler()
scaler.fit(features)
features = scaler.transform(features)

In [ ]:
# check if scaling was done
features[0]

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
LabelEncoder = LabelEncoder()
LabelEncoder.fit(labels)
labels = LabelEncoder.transform(labels)

In [ ]:
# Replacing the labels with integers
np.unique(labels)

In [ ]:
# Check if reverse encoding works
check = LabelEncoder.inverse_transform(labels)
check = pd.Series(check)
check.unique()

### Performing cross validation for neural network with 1 layer

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model():
    # Compile model
    dnn1= Sequential()
    dnn1.add(Dense(128,input_dim=77 ,activation='relu'))
    dnn1.add(Dropout(0.1))
    dnn1.add(Dense(15, activation='softmax'))
    
    # Compile model
    dnn1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return dnn1


In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# create model
model = KerasClassifier( build_fn=create_model, epochs= 30, batch_size=128, verbose=0)

In [ ]:
# 5-fold cross validation
# StratifiedKFold is a variation of Kfold that first shuffles the data with a given random state
kfold = StratifiedKFold(n_splits= 5, shuffle=True, random_state=seed)

In [ ]:
# Printing the mean accuracy value for the 5 folds
results = cross_val_score(model, features, labels, cv=kfold,scoring='accuracy', error_score="raise")
print(results.mean())

In [ ]:
results

# Performing cross validation for deep neural network with 2 layers

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_dnn2():
    # Compile model
    model= Sequential()
    model.add(Dense(128,input_dim=77 ,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(15, activation='softmax'))
    
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
# create model
dnn2 = KerasClassifier( build_fn=create_model_dnn2, epochs= 30, batch_size=128, verbose=0)

In [ ]:
# Printing the mean accuracy value for the 5 folds
results_dnn2 = cross_val_score(dnn2, features, labels, cv=kfold,scoring='accuracy', error_score="raise")
print(results_dnn2.mean())

In [ ]:
results_dnn2

### Performing cross validation for deep neural network with 3 layers

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_dnn3():
    # Compile model
    model= Sequential()
    model.add(Dense(256,input_dim=77 ,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(15, activation='softmax'))
    
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
# create model
dnn3 = KerasClassifier( build_fn=create_model_dnn2, epochs= 30, batch_size=256, verbose=0)

In [ ]:
# Printing the mean accuracy value for the 5 folds
results_dnn3 = cross_val_score(dnn3, features, labels, cv=kfold,scoring='accuracy', error_score="raise")
print(results_dnn3.mean())
results_dnn2

### Performing cross validation for convolutional neural network with 1 layer

In [ ]:
y_train= to_categorical(labels)
X_train = np.array(features).reshape(features.shape[0], features.shape[1], 1)

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_cnn1():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    cnn1 = Sequential()
    cnn1.add(Conv1D(64, 3,input_shape=(77, 1) ))
    cnn1.add(LeakyReLU(alpha=0.1))
    cnn1.add(MaxPooling1D(pool_size=2))
    cnn1.add(Dropout(0.3))

    cnn1.add(Flatten())
    cnn1.add(Dense(64, input_dim=78, kernel_initializer=hidden_initializer))
    cnn1.add(LeakyReLU(alpha=0.1))
    cnn1.add(Dense(32))
    cnn1.add(LeakyReLU(alpha=0.1))
    cnn1.add(Dense(15, activation='sigmoid'))
    
    # Compile model
    cnn1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return cnn1


In [ ]:
# create model
cnn1 = KerasClassifier( build_fn=create_model_cnn1, epochs= 30, batch_size=1024, verbose=0)

In [ ]:
# Printing the mean accuracy value for the 5 folds
results_cnn1 = cross_val_score(cnn1, X_train, labels, cv=kfold,scoring='accuracy', error_score="raise")
print(results_cnn1.mean())
results_cnn1

### Performing cross validation for convolutional neural network with 2 layers

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_cnn2():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    cnn2 = Sequential()
    cnn2.add(Conv1D(64, 3,input_shape=(77, 1) ))
    cnn2.add(LeakyReLU(alpha=0.1))
    cnn2.add(MaxPooling1D(pool_size=2))
    cnn2.add(Dropout(0.3))

    cnn2.add(Conv1D(32, 1))
    cnn2.add(LeakyReLU(alpha=0.1))
    cnn2.add(MaxPooling1D(pool_size=2))
    cnn2.add(Dropout(0.3))

    cnn2.add(Flatten())
    cnn2.add(Dense(64, input_dim=77, kernel_initializer=hidden_initializer))
    cnn2.add(LeakyReLU(alpha=0.1))
    cnn2.add(Dense(32))
    cnn2.add(LeakyReLU(alpha=0.1))
    cnn2.add(Dense(15, activation='sigmoid'))
    
    # Compile model
    cnn2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return cnn2


In [ ]:
# create model
cnn2 = KerasClassifier(build_fn=create_model_cnn2, epochs= 30, batch_size=1024, verbose=0)

In [ ]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_cnn2 = cross_val_score(cnn2, X_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_cnn2.mean())
results_cnn2

### Performing cross validation for convolutional neural network with 3 layers

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_cnn3():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    cnn3 = Sequential()
    cnn3.add(Conv1D(64, 3,input_shape=(77, 1) ))
    cnn3.add(LeakyReLU(alpha=0.1))
    cnn3.add(MaxPooling1D(pool_size=2))
    cnn3.add(Dropout(0.3))

    cnn3.add(Conv1D(64, 1))
    cnn3.add(LeakyReLU(alpha=0.1))
    cnn3.add(MaxPooling1D(pool_size=2))
    cnn3.add(Dropout(0.4))

    cnn3.add(Conv1D(64, 1))
    cnn3.add(LeakyReLU(alpha=0.1))
    cnn3.add(MaxPooling1D(pool_size=2))
    cnn3.add(Dropout(0.4))


    cnn3.add(Flatten())
    cnn3.add(Dense(64, input_dim=77, kernel_initializer=hidden_initializer))
    cnn3.add(LeakyReLU(alpha=0.1))
    cnn3.add(Dense(32))
    cnn3.add(LeakyReLU(alpha=0.1))
    cnn3.add(Dense(15, activation='sigmoid'))
    
    # Compile model
    cnn3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return cnn3


In [ ]:
# create model
cnn3 = KerasClassifier( build_fn=create_model_cnn3, epochs= 30, batch_size=512, verbose=0)

In [ ]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_cnn3 = cross_val_score(cnn3, X_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_cnn3.mean())
results_cnn3

### Performing cross validation for recurrent neural network with 1 layer

In [ ]:
ft_train = np.array(features).reshape(features.shape[0], features.shape[1], 1)
ft_labels= to_categorical(labels)

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_rnn1():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    # Initialising the RNN
    rnn1 = Sequential()

    # Adding the first RNN layer and some Dropout regularisation
    rnn1.add(SimpleRNN(units = 64,input_shape=(77, 1), activation='relu', return_sequences = True,))
    rnn1.add(Dropout(0.1))

    # Adding the output layer
    rnn1.add(Flatten())
    rnn1.add(Dense(64, input_dim=77, kernel_initializer=hidden_initializer))
    rnn1.add(LeakyReLU(alpha=0.1))
    rnn1.add(Dense(32))
    rnn1.add(LeakyReLU(alpha=0.1))
    rnn1.add(Dense(units = 15, activation='sigmoid'))
    
    # Compile model
    rnn1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return rnn1


In [ ]:
# create model
rnn1 = KerasClassifier( build_fn=create_model_rnn1, epochs= 30, batch_size=512, verbose=0)

In [ ]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_rnn1 = cross_val_score(rnn1, ft_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_rnn1.mean())

### Performing cross validation for recurrent neural network with 2 layers

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_rnn2():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    # Initialising the RNN
    rnn2 = Sequential()

    # Adding the first RNN layer and some Dropout regularisation
    rnn2.add(SimpleRNN(units = 64,input_shape=(77, 1), activation='relu', return_sequences = True,))
    rnn2.add(Dropout(0.1))
    
    # Adding the second RNN layer and some Dropout regularisation
    rnn2.add(SimpleRNN(units = 64, activation='relu', return_sequences = True,))
    rnn2.add(Dropout(0.1))

    # Adding the output layer
    rnn2.add(Flatten())
    rnn2.add(Dense(64, input_dim=77, kernel_initializer=hidden_initializer))
    rnn2.add(LeakyReLU(alpha=0.1))
    rnn2.add(Dense(32))
    rnn2.add(LeakyReLU(alpha=0.1))
    rnn2.add(Dense(units = 15, activation='sigmoid'))
    
    # Compile model
    rnn2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return rnn2


In [ ]:
# create model
rnn2 = KerasClassifier( build_fn=create_model_rnn2, epochs= 30, batch_size=1024, verbose=0)

In [ ]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_rnn2 = cross_val_score(rnn2, ft_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_rnn2.mean())

In [ ]:
results_rnn2

### Performing cross validation for recurrent neural network with 3 layers

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_rnn3():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    # Initialising the RNN
    rnn3 = Sequential()

    # Adding the first RNN layer and some Dropout regularisation
    rnn3.add(SimpleRNN(units = 64,activation='relu', return_sequences = True,  input_shape = (77,1)))
    rnn3.add(Dropout(0.1))

    # Adding the first RNN layer and some Dropout regularisation
    rnn3.add(SimpleRNN(units = 64,activation='relu', return_sequences = True,  ))
    rnn3.add(Dropout(0.1))
    
    # Adding a third RNN layer and some Dropout regularisation
    rnn3.add(SimpleRNN(units = 64,activation='relu', return_sequences = True))
    rnn3.add(Dropout(0.1))

    # Adding the output layer
    rnn3.add(Flatten())
    rnn3.add(Dense(64, input_dim=77, kernel_initializer=hidden_initializer))
    rnn3.add(LeakyReLU(alpha=0.1))
    rnn3.add(Dense(32))
    rnn3.add(LeakyReLU(alpha=0.1))
    rnn3.add(Dense(units = 15, activation='sigmoid'))
    
    # Compile model
    rnn3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return rnn3


In [ ]:
# create model
rnn3 = KerasClassifier( build_fn=create_model_rnn3, epochs= 30, batch_size=4096, verbose=0)

In [ ]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_rnn3 = cross_val_score(rnn3, ft_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_rnn3.mean())

In [ ]:
results_rnn3